<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


In [3]:
yelp['text']

0       BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...
1       Came here for lunch Togo. Service was quick. S...
2       I've been to Vegas dozens of times and had nev...
3       We went here on a night where they closed off ...
4       3.5 to 4 stars\n\nNot bad for the price, $12.9...
                              ...                        
9995    My family and I were hungry and this Subway is...
9996    My wife and I came here with a a couple of fri...
9997    The food was just OK and not anything to brag ...
9998    Today's visit is great!! Love and enjoy Town S...
9999    This is the absolute worst place I have ever s...
Name: text, Length: 10000, dtype: object

## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [40]:
# I chose to use SpaCy!

from spacy.lang.en import English
nlp = English()
tokenizer = nlp.Defaults.create_tokenizer(nlp)

In [ ]:
import spacy
from spacy.tokenizer import Tokenizer

nlp = spacy.load("en_core_web_lg")

# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

In [245]:
STOP_WORDS = nlp.Defaults.stop_words.union(['\n\n', '\n', ' ', 'service', 'good', 'food', 'place', 'like', 'just', 'really', 'got', 'don', 've', 'did', 'didn', 'great', 'time', '$', 'little', 'come', 'came', 'best', 'nice', 'love', 'order', 'ordered'])

In [231]:
def tokenize(doc):
    tokens_new = []
    tokens = tokenizer(doc)
    for token in tokens:
        if (token.is_punct == False) and (token.text.lower() not in STOP_WORDS):
            tokens_new.append(token.text.lower())
    return tokens_new

In [246]:
text_tokens = yelp['text'].apply(tokenize)

In [247]:
yelp['text_tokens'] = text_tokens

In [248]:
yelp['text_tokens'].head(20)

0     [beware, fake, fake, fake, small, business, lo...
1     [lunch, togo, quick, staff, friendly, complain...
2     [vegas, dozens, times, stepped, foot, circus, ...
3     [went, night, closed, street, party, actually,...
4     [3.5, 4, stars, bad, price, 12.99, lunch, seni...
5     [tasty, fast, casual, latin, street, menu, ove...
6     [absolutely, amazing, incredible, production, ...
7     [pho, enjoyed, 9:00pm, busy, served, right, aw...
8     [absolutely, unique, experience, nail, shop, f...
9     [wow, walked, sat, bar, 10, minutes, bartender...
10    [popped, dinner, yesterday, reservation, despi...
11    [thw, worst, stay, ended, paying, 700, 800, 2,...
12    [friendly, customer, quality, waitress, checke...
13    [super, busy, server, attentive, try, bacon, f...
14    [talk, getting, ripped, charged, 420, shut, va...
15    [girls, night, tonight, kid, decided, drive, h...
16    [stopped, drinks, flying, charlotte, weeks, ba...
17    [excellent, restaurant, encourage, visitin

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [85]:
# Starting with a little exploration...

vect = CountVectorizer(stop_words='english')

vect.fit(yelp['text'])

dtm = vect.transform(yelp['text'])

In [86]:
dtm = pd.DataFrame(dtm.todense(), columns=vect.get_feature_names())

In [87]:
wc_top20 = pd.DataFrame(dtm.sum().sort_values(ascending=False).head(20),
                       columns=['count'])

In [88]:
wc_top20

,count
food,5070
place,4832
good,4806
great,4297
service,3695
like,3558
just,3553
time,3449
really,2692
got,2070


Use TF-IDF to vectorize my words...

In [89]:
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

dtm = tfidf.fit_transform(yelp['text'])

dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

In [76]:
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=10000,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<spacy.tokenizer.Tokenizer object at 0x0000024EE44DAD68>,
                use_idf=True, vocabulary=None)

In [90]:
dtm.head()

,00,000,00pm,07,10,100,1000,101,10pm,11,...,yuck,yuk,yum,yummy,yup,zero,zone,zoo,zucchini,était
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Fit a NearestNeighbors model

In [91]:
nn = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [99]:
review = ['Wonderful burrito, great coffee, nice owner, convenient parking.']

In [100]:
new = tfidf.transform(review)

In [101]:
new.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.]])

Find matching reviews!

In [108]:
matches = nn.kneighbors(new.todense(), n_neighbors=10)[1]

In [109]:
for match in matches:
    print(yelp['text'][match])

6311    天氣很熱吃不下東西，今天我點了一個韓國冷面湯、餐後點了甜點，冰沙系列不會太甜膩，覺得店家很用...
6204    旅行でラスベガスに来ましたがネイルがはげてるのが気になり、探したお店でした。\n質問にも丁寧...
6029    Just discovered this place, a hidden treasure ...
4867    The ambiance of this place is a 4 or a 5- it's...
4536    Burritos are on the "smaller" side but still g...
5784    This is a great location. It's not as packed a...
6290    Great place for a decent burrito.\nLove the ca...
721     ordered a spicy bbq steak burrito. add topping...
4094    It's wonderful to find a coffee shop that care...
822     Bomb tacos! Menu is simplistic, but food is al...
Name: text, dtype: object


Okay, I truly don't know why these reviews in Japanese and Chinese are matching at the top there, but the rest of the reviews seem to match my contrived sample pretty well.

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [110]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [111]:
vect = TfidfVectorizer(stop_words='english')
clf = RandomForestClassifier()

pipe = Pipeline([('vect', vect), ('clf', clf)])

In [114]:
yelp.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id', 'text_tokens'],
      dtype='object')

In [115]:
pipe.fit(yelp['text'], yelp['stars'])

Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words='english', strip_accents=None,
                                 sublinear_tf=False,
                                 token_patte...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=None, max_features='auto',
                                        max_leaf_nodes=None, max_samples=None

In [120]:
bad_review = ['This place stinks, plain and simple. Worst calamari ever.']

In [119]:
pipe.predict(review)

array([5], dtype=int64)

In [121]:
pipe.predict(bad_review)

array([1], dtype=int64)

The predictor seems to work!

In [122]:
parameters = {
    'vect__max_df': (0.8, 0.9, 1.0),
    'vect__min_df': (.02, .05, .1),
#     'vect__max_features': (500,5000),
    'clf__n_estimators': (50, 100),
    'clf__max_depth': (20, 100, 200)
}

grid_search = GridSearchCV(pipe, parameters, cv=3, n_jobs=4, verbose=1)
grid_search.fit(yelp['text'], yelp['stars'])

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:   49.9s
[Parallel(n_jobs=4)]: Done 162 out of 162 | elapsed:  5.5min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        no

In [123]:
grid_search.best_params_

{'clf__max_depth': 100,
 'clf__n_estimators': 100,
 'vect__max_df': 0.8,
 'vect__min_df': 0.02}

In [124]:
grid_search.predict(review)

array([5], dtype=int64)

In [125]:
grid_search.predict(bad_review)

array([1], dtype=int64)

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [129]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

Learn the vocubalary of the yelp data:

In [249]:
id2word = Dictionary(yelp['text_tokens'])

In [250]:
len(id2word.keys())

29025

In [251]:
id2word.filter_extremes(no_below=5, no_above=0.95)

In [252]:
len(id2word.keys())

7706

Create a bag of words representation of the entire corpus

In [253]:
corpus = [id2word.doc2bow(tokens) for tokens in text_tokens]

Your LDA model should be ready for estimation: 

In [270]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 3 # You can change this parameter
                  )

In [271]:
lda.print_topics()

[(0,
  '0.004*"staff" + 0.004*"people" + 0.003*"pretty" + 0.003*"definitely" + 0.003*"day" + 0.003*"restaurant" + 0.003*"way" + 0.003*"recommend" + 0.003*"friendly" + 0.003*"went"'),
 (1,
  '0.005*"definitely" + 0.004*"know" + 0.004*"amazing" + 0.004*"staff" + 0.004*"menu" + 0.004*"friendly" + 0.004*"try" + 0.004*"experience" + 0.004*"restaurant" + 0.004*"people"'),
 (2,
  '0.004*"went" + 0.004*"try" + 0.004*"chicken" + 0.004*"said" + 0.004*"delicious" + 0.003*"staff" + 0.003*"experience" + 0.003*"restaurant" + 0.003*"people" + 0.003*"friendly"')]

Create 1-2 visualizations of the results

In [226]:
import re

In [272]:
words = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

In [273]:
topics = [' '.join(t[0:3]) for t in words]

In [274]:
topics

['staff people pretty', 'definitely know amazing', 'went try chicken']

In [275]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [276]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.005100 -0.000835       1        1  33.497429
2      0.003305 -0.003813       2        1  33.454178
0      0.001795  0.004648       3        1  33.048386, topic_info=           Term         Freq        Total Category  logprob  loglift
453        know  1208.000000  1208.000000  Default  30.0000  30.0000
125         bad   827.000000   827.000000  Default  29.0000  29.0000
289         try  1437.000000  1437.000000  Default  28.0000  28.0000
30         said  1194.000000  1194.000000  Default  27.0000  27.0000
167        menu  1222.000000  1222.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
347       fresh   306.235138   935.551025   Topic3  -6.0938  -0.0096
180     amazing   341.592346  1346.144287   Topic3  -5.9845  -0.2642
192       going   321.032593  1141.997925   Topic3  -6.0466  -0.1618
245  experience   327.434540  1286.191772   Topic3  -6.0268  -0.2610
289         try   325.092712  1437.641846   Topic3  -6.0340  -0.3795

[288 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
1046      1  0.333601   \n\n
1046      2  0.282150   \n\n
1046      3  0.385052   \n\n
1147      1  0.455393       
1147      2  0.335795       
...     ...       ...    ...
7388      2  0.276169   zach
7388      3  0.276169   zach
2558      1  0.476596   zero
2558      2  0.251537   zero
2558      3  0.278015   zero

[624 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1])

### Analysis

The tokens seem to split pretty well into 3 or 4 groups. I see topics for the overall experience a restaurant offers, the service, and the food. It would be interesting to get user ratings of each of those subcategories and then train a model specifically to predict food rating for a review. The other thing I notice from these visualizations is that people really like mentioning the chicken in their yelp reviews, for whatever reason.

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)